<a href="https://colab.research.google.com/github/jovansx/siap-project/blob/develop/siap-script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

# Methods

In [183]:
def convert_champion_json_to_map():
  df_champs = pd.read_json('/content/drive/MyDrive/SIAP Data/champion_info.json')
  champ_map = {}

  for ch in df_champs.data:
    champ_map[ch["id"]] = ch["name"]
  return champ_map

def generate_df2(champ_map, champ_id, champ_name, df1):
  df2 = pd.DataFrame(list(champ_map.items()), columns = [champ_id,champ_name])
  merged = pd.merge(df1, df2, on=champ_id)
  dropped = merged.drop(columns=[champ_id], axis=1)
  return dropped

def merge(champ_map, df1):
  df1 = generate_df2(champ_map, 't1_champ1id', 't1_champ1name', df1)
  df1 = generate_df2(champ_map, 't1_champ2id', 't1_champ2name', df1)
  df1 = generate_df2(champ_map, 't1_champ3id', 't1_champ3name', df1)
  df1 = generate_df2(champ_map, 't1_champ4id', 't1_champ4name', df1)
  df1 = generate_df2(champ_map, 't1_champ5id', 't1_champ5name', df1)
  df1 = generate_df2(champ_map, 't2_champ1id', 't2_champ1name', df1)
  df1 = generate_df2(champ_map, 't2_champ2id', 't2_champ2name', df1)
  df1 = generate_df2(champ_map, 't2_champ3id', 't2_champ3name', df1)
  df1 = generate_df2(champ_map, 't2_champ4id', 't2_champ4name', df1)
  df1 = generate_df2(champ_map, 't2_champ5id', 't2_champ5name', df1)
  return df1

def delete_columns(df, columns_deletion):
  return df.drop(columns=columns_deletion, axis=1)

def preprocess_dataframe(df_stats):
  df_stats = delete_columns(df_stats, ["Class", "Role", "Tier", "Trend", "Role %", "Pick %", "Ban %", "KDA"])   # Remove columns
  df_stats['Win %'] = df_stats['Win %'].str[:5].astype(float)                                                   # Remove % and convert to float
  distinct_names = set(df_stats['Name'])                                                                        # Distinct names
  new_df_stats = pd.DataFrame({'Name': [], 'Score': [], 'Win': []})                                             # New empty dataframe of stats

  for name in distinct_names:
    sub_df_stats = df_stats.loc[df_stats['Name'] == name]
    score = sub_df_stats["Score"].mean(axis=0)
    win_rate = sub_df_stats["Win %"].mean(axis=0)
    one_row_df = pd.DataFrame({"Name": [name], "Score": [score], "Win": [win_rate]})
    new_df_stats = new_df_stats.append(one_row_df)                                                              # Append average values for every champion
  new_df_stats.index = range(1,len(new_df_stats)+1)                                                             # Set incremental indexes
  return new_df_stats

def merge_games_with_stats_helper(df_games, df_stats, old_score, new_score, old_win, new_win, name):
  df_stats.columns = df_stats.columns.str.replace(old_score, new_score)
  df_stats.columns = df_stats.columns.str.replace(old_win, new_win)
  df_games.columns = df_games.columns.str.replace(name, "Name")
  df_games = pd.merge(df_games, df_stats, on="Name")
  df_games = df_games.drop(columns=["Name"], axis=1)
  return df_games, df_stats

def merge_games_with_stats(df_games, df_stats):
  df_games, df_stats = merge_games_with_stats_helper(df_games, df_stats, "Score", "t1_champ1score", "Win", "t1_champ1win", "t1_champ1name")
  df_games, df_stats = merge_games_with_stats_helper(df_games, df_stats, "t1_champ1score", "t1_champ2score", "t1_champ1win", "t1_champ2win", "t1_champ2name")
  df_games, df_stats = merge_games_with_stats_helper(df_games, df_stats, "t1_champ2score", "t1_champ3score", "t1_champ2win", "t1_champ3win", "t1_champ3name")
  df_games, df_stats = merge_games_with_stats_helper(df_games, df_stats, "t1_champ3score", "t1_champ4score", "t1_champ3win", "t1_champ4win", "t1_champ4name")
  df_games, df_stats = merge_games_with_stats_helper(df_games, df_stats, "t1_champ4score", "t1_champ5score", "t1_champ4win", "t1_champ5win", "t1_champ5name")
  df_games, df_stats = merge_games_with_stats_helper(df_games, df_stats, "t1_champ5score", "t2_champ1score", "t1_champ5win", "t2_champ1win", "t2_champ1name")
  df_games, df_stats = merge_games_with_stats_helper(df_games, df_stats, "t2_champ1score", "t2_champ2score", "t2_champ1win", "t2_champ2win", "t2_champ2name")
  df_games, df_stats = merge_games_with_stats_helper(df_games, df_stats, "t2_champ2score", "t2_champ3score", "t2_champ2win", "t2_champ3win", "t2_champ3name")
  df_games, df_stats = merge_games_with_stats_helper(df_games, df_stats, "t2_champ3score", "t2_champ4score", "t2_champ3win", "t2_champ4win", "t2_champ4name")
  df_games, df_stats = merge_games_with_stats_helper(df_games, df_stats, "t2_champ4score", "t2_champ5score", "t2_champ4win", "t2_champ5win", "t2_champ5name")
  return df_games

# Main 

In [185]:

df_games = pd.read_csv('/content/drive/MyDrive/SIAP Data/games.csv')
df_stats = pd.read_csv('/content/drive/MyDrive/SIAP Data/League of Legends Champion Stats 12.1.csv', delimiter=';')

champ_map = convert_champion_json_to_map()
df_games = merge(champ_map, df_games)
df_games = delete_columns(df_games, ["gameId", "creationTime", "seasonId", "firstBlood", "firstTower", "firstInhibitor", "firstBaron",
                                     "firstDragon", "firstRiftHerald", "t1_towerKills", "t1_inhibitorKills", "t1_baronKills", "t1_dragonKills",
                                     "t1_riftHeraldKills", "t2_towerKills", "t2_inhibitorKills", "t2_baronKills", "t2_dragonKills", "t2_riftHeraldKills"])
df_games = delete_columns(df_games, ["t1_champ1_sum1", "t1_champ1_sum2", "t1_champ2_sum1", "t1_champ2_sum2", "t1_champ3_sum1", "t1_champ3_sum2",
                                     "t1_champ4_sum1", "t1_champ4_sum2", "t1_champ5_sum1", "t1_champ5_sum2", "t2_champ1_sum1", "t2_champ1_sum2",
                                     "t2_champ2_sum1", "t2_champ2_sum2", "t2_champ3_sum1", "t2_champ3_sum2", "t2_champ4_sum1", "t2_champ4_sum2",
                                     "t2_champ5_sum1", "t2_champ5_sum2"])

df_stats = preprocess_dataframe(df_stats)

df_games = merge_games_with_stats(df_games, df_stats)
print(df_games.columns)

# Uradi normalizaciju podataka

# Napravi model (Vise modela pa da poredimo)

# Treniraj

# Testiraj

Index(['gameDuration', 'winner', 't1_ban1', 't1_ban2', 't1_ban3', 't1_ban4',
       't1_ban5', 't2_ban1', 't2_ban2', 't2_ban3', 't2_ban4', 't2_ban5',
       't1_champ1score', 't1_champ1win', 't1_champ2score', 't1_champ2win',
       't1_champ3score', 't1_champ3win', 't1_champ4score', 't1_champ4win',
       't1_champ5score', 't1_champ5win', 't2_champ1score', 't2_champ1win',
       't2_champ2score', 't2_champ2win', 't2_champ3score', 't2_champ3win',
       't2_champ4score', 't2_champ4win', 't2_champ5score', 't2_champ5win'],
      dtype='object')
